In [114]:
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model, datasets
from helpers import *

In [115]:
timestep=0.128040973111396;
fc=1/timestep;

In [116]:
yes_contents = sio.loadmat('NIRS_yes_signal.mat')
no_contents = sio.loadmat('NIRS_no_signal.mat')

In [117]:
def feature_extraction(signal,feature_dictionary):
    feature=[]
    if(feature_dictionary["fft_max_frequencies"]==1):
        f=np.linspace(0, fc, num=signal.size)
        spectrum= np.abs(np.fft.fft(signal));
        fft_max_frequencies=f[np.argsort(spectrum)[-3:-1]]
        feature=np.concatenate((feature,fft_max_frequencies), axis=0);

    if(feature_dictionary["mean"]==1):
        mean=np.mean(signal);
        feature=np.concatenate((feature,[mean]), axis=0)
        
    if(feature_dictionary["variance"]==1):
        variance=np.var(signal)
        feature=np.concatenate((feature,[mean]), axis=0)
        
    return np.asarray(feature)
#.reshape([feature.size,1])
        
    

In [118]:
def get_train_matrix(channels, feature_dictionary,label):
    list_train=[]
    list_labels=[]
    dim=channels.shape
    for i in range (0,dim[1]):
        single_measurement=channels[0,i]
        dim1=single_measurement.shape
        for j  in range (0,dim1[1]):
            features=feature_extraction(single_measurement[:,j],feature_dictionary)
            list_train.append([features]);
        
        labels=get_labels(dim1[1],label);
        list_labels.append([labels]);
        
            
            
    train_TX=np.concatenate(list_train)
    labels=np.concatenate(list_labels,axis=1)
    
    return train_TX,labels.T.reshape(labels.size)

def get_labels(number, string):
    if(string=="No"):
        return np.zeros(number)    
    if(string=="Yes"):
        return np.ones(number)
    

In [119]:
#buildig the train matrix and labels

channels_no=no_contents["no_signal"]
channels_yes=yes_contents["yes_signal"]

# select which feature select
feature_dictionary = {
        "fft_max_frequencies" : 1, 
         "mean" : 1, 
         "autocorrelation" : 1,
         "variance" : 1
         }


train_TX_yes,labels_yes=get_train_matrix(channels_yes, feature_dictionary,"Yes");
train_TX_no,labels_no=get_train_matrix(channels_no, feature_dictionary,"No");

train_TX=np.concatenate((train_TX_yes,train_TX_no),axis=0)
labels=np.concatenate((labels_yes,labels_no),axis=0)

In [120]:
#logistic regression


logreg = linear_model.LogisticRegression(C=1e5)
logreg.fit(train_TX, labels)

LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)